In [1]:
%pip install pysmiles

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pandas as pd
from pysmiles import read_smiles
import networkx as nx
import matplotlib.pyplot as plt

## ESOL

In [3]:
df = pd.read_csv("../../kaggle/sources/delaney-processed.csv", sep='\t' if '\t' in open("../../kaggle/sources/delaney-processed.csv").readline() else ',')

polymers_root = "../../kaggle/ESOL"
os.makedirs(polymers_root, exist_ok=True)

error_ids = []

for index, row in df.iterrows():
    polymer_id = str(row['Compound ID'])
    print(polymer_id)
    smiles = row['smiles']

    if pd.isna(smiles) or not isinstance(smiles, str) or smiles.strip() == "":
        print(f"SKIP {polymer_id}: SMILES is missing")
        error_ids.append(polymer_id)
        continue

    try:
        mol = read_smiles(smiles)

        polymer_dir = os.path.join(polymers_root, polymer_id)
        os.makedirs(polymer_dir, exist_ok=True)

        adj = nx.to_numpy_array(mol)
        pd.DataFrame(adj).to_csv(os.path.join(polymer_dir, "adjacency.csv"), index=False)

        atom_dict = nx.get_node_attributes(mol, "element")
        atom_df = pd.DataFrame(list(atom_dict.items()), columns=["node", "element"])
        atom_df.to_csv(os.path.join(polymer_dir, "atoms.csv"), index=False)

        pos = nx.spring_layout(mol)
        #plt.figure(figsize=(6, 6))
        #nx.draw(mol, with_labels=True, labels=atom_dict, pos=pos, node_size=500, font_size=8)
        #plt.gca().set_aspect('equal')
        #plt.axis('off')
        #plt.tight_layout()
        #plt.savefig(os.path.join(polymer_dir, "molecule.png"))
        #plt.close()

        details_cols = ['ESOL predicted log solubility in mols per litre']
        details_df = row[details_cols].to_frame().T
        details_df.to_csv(os.path.join(polymer_dir, "details.csv"), index=False)


        print(f"Processed {polymer_id}")

    except Exception as e:
        print(f"ERROR processing {polymer_id}: {e}")
        error_ids.append(polymer_id)


pd.DataFrame(error_ids, columns=["id"]).to_csv(os.path.join(polymers_root, "Falseparse.csv"), index=False)

Amigdalin
Processed Amigdalin
Fenfuram
Processed Fenfuram
citral
Processed citral
Picene
Processed Picene
Thiophene
Processed Thiophene
benzothiazole
Processed benzothiazole
2,2,4,6,6'-PCB
Processed 2,2,4,6,6'-PCB
Estradiol
Processed Estradiol
Dieldrin
Processed Dieldrin
Rotenone
Processed Rotenone
2-pyrrolidone
Processed 2-pyrrolidone
2-Chloronapthalene
Processed 2-Chloronapthalene
1-Pentene 
Processed 1-Pentene 
Primidone
Processed Primidone
Tetradecane
Processed Tetradecane
2-Chloropropane
Processed 2-Chloropropane
2-Methylbutanol
Processed 2-Methylbutanol
Benzonitrile
Processed Benzonitrile
Diazinon
Processed Diazinon
2-Undecanol
Processed 2-Undecanol
2,2',3,4,6-PCB
Processed 2,2',3,4,6-PCB
Lenacil
Processed Lenacil
Phorate
Processed Phorate
Phenacetin
Processed Phenacetin
Dinitramine
Processed Dinitramine
1-Heptanol
Processed 1-Heptanol
Theophylline
Processed Theophylline
Butethal
Processed Butethal
P,P'-DDE
Processed P,P'-DDE
Methyl octanoate
Processed Methyl octanoate
1,4-Diethy

## Lipophilicity

In [8]:
df = pd.read_csv("../../kaggle/sources/Lipophilicity.csv", sep='\t' if '\t' in open("../../kaggle/sources/Lipophilicity.csv").readline() else ',')

polymers_root = "../../kaggle/Lipophilicity"
os.makedirs(polymers_root, exist_ok=True)

error_ids = []

for index, row in df.iterrows():
    polymer_id = str(row['CMPD_CHEMBLID'])
    print(polymer_id)
    smiles = row['smiles']

    if pd.isna(smiles) or not isinstance(smiles, str) or smiles.strip() == "":
        print(f"SKIP {polymer_id}: SMILES is missing")
        error_ids.append(polymer_id)
        continue

    try:
        mol = read_smiles(smiles)

        polymer_dir = os.path.join(polymers_root, polymer_id)
        os.makedirs(polymer_dir, exist_ok=True)

        adj = nx.to_numpy_array(mol)
        pd.DataFrame(adj).to_csv(os.path.join(polymer_dir, "adjacency.csv"), index=False)

        atom_dict = nx.get_node_attributes(mol, "element")
        atom_df = pd.DataFrame(list(atom_dict.items()), columns=["node", "element"])
        atom_df.to_csv(os.path.join(polymer_dir, "atoms.csv"), index=False)

        pos = nx.spring_layout(mol)
        #plt.figure(figsize=(6, 6))
        #nx.draw(mol, with_labels=True, labels=atom_dict, pos=pos, node_size=500, font_size=8)
        #plt.gca().set_aspect('equal')
        #plt.axis('off')
        #plt.tight_layout()
        #plt.savefig(os.path.join(polymer_dir, "molecule.png"))
        #plt.close()

        details_cols = ['exp']
        details_df = row[details_cols].to_frame().T  # sorból DataFrame (1 sor)
        details_df.to_csv(os.path.join(polymer_dir, "details.csv"), index=False)


        print(f"Processed {polymer_id}")

    except Exception as e:
        print(f"ERROR processing {polymer_id}: {e}")
        error_ids.append(polymer_id)


pd.DataFrame(error_ids, columns=["id"]).to_csv(os.path.join(polymers_root, "Falseparse.csv"), index=False)

CHEMBL596271
Processed CHEMBL596271
CHEMBL1951080
Processed CHEMBL1951080
CHEMBL1771
Processed CHEMBL1771
CHEMBL234951
Processed CHEMBL234951
CHEMBL565079
Processed CHEMBL565079
CHEMBL317462
Processed CHEMBL317462
CHEMBL1951182
Processed CHEMBL1951182
CHEMBL2030960
Processed CHEMBL2030960
CHEMBL13097
Processed CHEMBL13097
CHEMBL1527751
Processed CHEMBL1527751
CHEMBL1940306
Processed CHEMBL1940306
CHEMBL578201
Processed CHEMBL578201
CHEMBL276218
Processed CHEMBL276218
CHEMBL2153181
Processed CHEMBL2153181
CHEMBL1916276
Processed CHEMBL1916276
CHEMBL93884
Processed CHEMBL93884
CHEMBL2171986
Processed CHEMBL2171986
CHEMBL190044
Processed CHEMBL190044
CHEMBL469790
Processed CHEMBL469790
CHEMBL2165055
Processed CHEMBL2165055
CHEMBL322479
Processed CHEMBL322479
CHEMBL2036736
Processed CHEMBL2036736
CHEMBL2179291
Processed CHEMBL2179291
CHEMBL2158823
Processed CHEMBL2158823
CHEMBL641
Processed CHEMBL641
CHEMBL1289785
Processed CHEMBL1289785
CHEMBL1381135
Processed CHEMBL1381135
CHEMBL528217
P

## NeurIPS Polimer Challenge 2025

In [11]:
df = pd.read_csv("../../kaggle/input/train.csv", sep='\t' if '\t' in open("../../kaggle/input/train.csv").readline() else ',')

polymers_root = "../../kaggle/NeurIPS"
os.makedirs(polymers_root, exist_ok=True)

error_ids = []

for index, row in df.iterrows():
    polymer_id = str(row['id'])
    print(polymer_id)
    smiles = row['SMILES']

    if pd.isna(smiles) or not isinstance(smiles, str) or smiles.strip() == "":
        print(f"SKIP {polymer_id}: SMILES is missing")
        error_ids.append(polymer_id)
        continue

    try:
        mol = read_smiles(smiles)

        polymer_dir = os.path.join(polymers_root, polymer_id)
        os.makedirs(polymer_dir, exist_ok=True)

        adj = nx.to_numpy_array(mol)
        pd.DataFrame(adj).to_csv(os.path.join(polymer_dir, "adjacency.csv"), index=False)

        atom_dict = nx.get_node_attributes(mol, "element")
        atom_df = pd.DataFrame(list(atom_dict.items()), columns=["node", "element"])
        atom_df.to_csv(os.path.join(polymer_dir, "atoms.csv"), index=False)

        pos = nx.spring_layout(mol)
        #plt.figure(figsize=(6, 6))
        #nx.draw(mol, with_labels=True, labels=atom_dict, pos=pos, node_size=500, font_size=8)
        #plt.gca().set_aspect('equal')
        #plt.axis('off')
        #plt.tight_layout()
        #plt.savefig(os.path.join(polymer_dir, "molecule.png"))
        #plt.close()

        details_cols = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
        details_df = row[details_cols].to_frame().T  
        details_df.to_csv(os.path.join(polymer_dir, "details.csv"), index=False)


        print(f"Processed {polymer_id}")

    except Exception as e:
        print(f"ERROR processing {polymer_id}: {e}")
        error_ids.append(polymer_id)

pd.DataFrame(error_ids, columns=["id"]).to_csv(os.path.join(polymers_root, "Falseparse.csv"), index=False)



87817
Processed 87817
106919
Processed 106919
388772
Processed 388772
519416
Processed 519416
539187
Processed 539187
950661
Processed 950661
1012066
Processed 1012066
1022104
Processed 1022104
1722264
Processed 1722264
2265305
Processed 2265305
2986007
Processed 2986007
3013292
Processed 3013292
3053950
Processed 3053950
3128201
Processed 3128201
3139746
Processed 3139746
3254580
Processed 3254580
4600132
Processed 4600132
4658296
Processed 4658296
4834929
Processed 4834929
5118841
Processed 5118841
5350166
Processed 5350166
5650517
Processed 5650517
5661484
Processed 5661484
5709902
Processed 5709902
5724071
Processed 5724071
5857263
Processed 5857263
5905810
Processed 5905810
5966923
Processed 5966923
6019988
Processed 6019988
6102563
Processed 6102563
6259923
Processed 6259923
6645418
Processed 6645418
6665789
Processed 6665789
7586800
Processed 7586800
7687820
Processed 7687820
8689461
Processed 8689461
8973725
Processed 8973725
9231170
Processed 9231170
9284896
Processed 9284896
